In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 127kB/s 
     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 6.7MB 34.8MB/s 
     |████████████████████████████████| 983kB 35.7MB/s 
     |████████████████████████████████| 245kB 38.2MB/s 
     |████████████████████████████████| 675kB 34.8MB/s 
     |████████████████████████████████| 471kB 33.1MB/s 
     |████████████████████████████████| 450kB 37.1MB/s 
     |████████████████████████████████| 3.9MB 32.6MB/s 
     |████████████████████████████████| 1.1MB 36.3MB/s 
     |████████████████████████████████| 890kB 35.6MB/s 
     |████████████████████████████████| 3.8MB 33.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.16.3-cp36-none-any.whl size=25246180 sha256=40883ad6757252d77bc6a4ac747d744c91a01b0ac2ef47521148dd0e69d8a200
  Stored in directory: /root/.cache/pip/wheels/2b/24/21/a52cc4dcb0b438cf570e9d44c017761d41335d9af4c189ebad
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-no

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
import numpy as np
import ktrain
import tensorflow as tf
import os.path
from ktrain import text

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
4.204363070999989
GPU (s):
0.18901582000000872
GPU speedup over CPU: 22x


In [5]:
dataset = tf.keras.utils.get_file(fname = "aclImdb_v1.tar.gz",
                                  origin = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
                                  extract = True) #extract is for ZIP File

84131840/84125825 [==============================] - 2s 0us/step


In [6]:
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [7]:
print(os.path.dirname(dataset))
print(IMDB_DATADIR)

/root/.keras/datasets
/root/.keras/datasets/aclImdb


In [8]:
(x_train, y_train), (x_test, y_test), preprocessing_mode = text.texts_from_folder(IMDB_DATADIR,
                                                                                  classes = ['pos', 'neg'],
                                                                                  maxlen = 500,
                                                                                  train_test_names = ['train', 'test'],
                                                                                  preprocess_mode = 'bert')

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [9]:
model = text.text_classifier(name = 'bert',
                             train_data = (x_train, y_train),
                             preproc = preprocessing_mode)

Is Multi-Label? False
maxlen is 500
done.


In [10]:
learner = ktrain.get_learner(model = model,
                             train_data = (x_train, y_train),
                             val_data = (x_test, y_test),
                             batch_size = 6)

In [11]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 9366s 375ms/sample - loss: 0.2586 - accuracy: 0.8892 - val_loss: 0.1674 - val_accuracy: 0.9375
